In [ ]:
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [70]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.llms import OpenAI
import pandas as pd
import numpy as np
import random
import time

In [ ]:
from secret_key import openapi_key

In [ ]:
from secret_key import openapi_key

os.environ['OPENAI_API_KEY'] = openapi_key


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/llm/data/Expanded_PROMISE.csv")

In [ ]:
df.head(2)

,Id,RequirementText,Class,Binary_Label
0,1,The product shall be available during normal b...,A,1
1,2,The product shall be available for use 24 hour...,A,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Id               969 non-null    int64 
 1   RequirementText  969 non-null    object
 2   Class            969 non-null    object
 3   Binary_Label     969 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 30.4+ KB


In [33]:
examples = []

for i in range(len(df["RequirementText"])):

    obj = {
        "text" : df["RequirementText"][i],
        "class" : "Functional" if df["Binary_Label"][i] == 0 else "Non Functional"
    }

    examples.append(obj)


random.shuffle(examples)

print(examples)

[{'text': 'Recreation of Inventory Quantity Adjustment for the period of previous 30 days shall take no more than 30min.', 'class': 'Non Functional'}, {'text': 'The estimator shall apply selected recycled parts to the collision estimate.', 'class': 'Functional'}, {'text': 'Velkat staff will be available for phone support between 7am-7pm  M-F to provide technical support to the Izogn administrators.', 'class': 'Non Functional'}, {'text': 'When the hit results in sinking a ship  the product shall mark the ship as sunk on the defensive grid of the defensive player.', 'class': 'Functional'}, {'text': 'Unauthorised users can only read data without modifying anything.', 'class': 'Non Functional'}, {'text': 'Each account must have a separate identifying account number.', 'class': 'Functional'}, {'text': 'The system shall be available for use between the hours of 8am and 6pm.', 'class': 'Non Functional'}, {'text': 'An anonymous survey will show that 95% of customers are comfortable using and p

In [34]:
prompt_template = '''
Text: {text}
{class}
'''

example_prompt = PromptTemplate(input_variables=["text","class"],template = prompt_template)

print(example_prompt.format(**examples[0]))


Text: Recreation of Inventory Quantity Adjustment for the period of previous 30 days shall take no more than 30min.
Non Functional



In [36]:
examples[:6]

[{'text': 'Recreation of Inventory Quantity Adjustment for the period of previous 30 days shall take no more than 30min.',
  'class': 'Non Functional'},
 {'text': 'The estimator shall apply selected recycled parts to the collision estimate.',
  'class': 'Functional'},
 {'text': 'Velkat staff will be available for phone support between 7am-7pm  M-F to provide technical support to the Izogn administrators.',
  'class': 'Non Functional'},
 {'text': 'When the hit results in sinking a ship  the product shall mark the ship as sunk on the defensive grid of the defensive player.',
  'class': 'Functional'},
 {'text': 'Unauthorised users can only read data without modifying anything.',
  'class': 'Non Functional'},
 {'text': 'Each account must have a separate identifying account number.',
  'class': 'Functional'}]

In [58]:



final_prompt = FewShotPromptTemplate(
    examples=examples[:6],
    example_prompt=example_prompt,
    suffix= "Text: {text}/n",
    input_variables=["text"],
    prefix= 'Classify the sentiment of the following text as "Funtional" or "Non Funtional"'
)



In [59]:
print(final_prompt.format(text =examples[3]["text"] ))

Classify the sentiment of the following text as "Funtional" or "Non Funtional"


Text: Recreation of Inventory Quantity Adjustment for the period of previous 30 days shall take no more than 30min.
Non Functional



Text: The estimator shall apply selected recycled parts to the collision estimate.
Functional



Text: Velkat staff will be available for phone support between 7am-7pm  M-F to provide technical support to the Izogn administrators.
Non Functional



Text: When the hit results in sinking a ship  the product shall mark the ship as sunk on the defensive grid of the defensive player.
Functional



Text: Unauthorised users can only read data without modifying anything.
Non Functional



Text: Each account must have a separate identifying account number.
Functional


Text: When the hit results in sinking a ship  the product shall mark the ship as sunk on the defensive grid of the defensive player./n


In [60]:
llm = OpenAI(temperature=0.2)

In [61]:
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.2, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [62]:
extraction_chain = LLMChain(llm = llm , prompt = final_prompt , output_key= "class" )

# Prediction

In [66]:
output = extraction_chain.predict(text =examples[50]["text"] )

In [67]:
print(output)


Non Functional


In [68]:
examples[50]["class"]

'Non Functional'

In [77]:
def preds(n,s = 30):

    for i in range(n,n+5):
        time.sleep(s)
        print("*"*100)
        print()
        print("Input Text")
        print(examples[i]["text"])
        print()
        print("Actual Class: ",examples[i]["class"] )
        print()
        print("Predicted: ", extraction_chain.predict(text =examples[i]["text"] ))
        print()




In [76]:
preds(50)

****************************************************************************************************

Input Text
The product shall have a conservative and professional appearance.

Actual Class  Non Functional

Predicted  
Non Functional

****************************************************************************************************

Input Text
The system shall display available payment methods for payment.

Actual Class  Functional

Predicted  
Non Functional

****************************************************************************************************

Input Text
100% of cardmember services representatives shall be able to successfully create a dispute case on the first encounter after completing the training course.

Actual Class  Non Functional

Predicted  Non Functional

****************************************************************************************************

Input Text
The system shall allow user to choose their preferable topping.

Actual Class  Functiona

In [78]:
preds(80)

****************************************************************************************************

Input Text
The online registration system is "self-service style"system that shall initially address the student registration needs.

Actual Class:  Functional

Predicted:  
Non Functional

****************************************************************************************************

Input Text
The meeting scheduler system shall be able to select a participant as an important participant.

Actual Class:  Functional

Predicted:  
Non Functional

****************************************************************************************************

Input Text
The product shall have audit capabilities.The product shall store messages for a minimum of one year for audit and transaction tracking purposes.

Actual Class:  Non Functional

Predicted:  
Functional

****************************************************************************************************

Input Text
Program Admin